In [2]:
# ============================================================
# Block A0 (robust) — Clean unmount + fresh Google Drive mount
# Why:
#   - Fixes "Mountpoint must not already contain files" errors.
#   - Ensures nothing wrote to /content/drive before mounting.
# Run this as the FIRST cell in the notebook.
# ============================================================

import os, shutil, subprocess, time

MOUNTPOINT = "/content/drive"

def _safe_unmount(mp=MOUNTPOINT):
    # 1) Try Colab's unmount (if previously mounted)
    try:
        from google.colab import drive as _d
        _d.flush_and_unmount()
    except Exception:
        pass

    # 2) Try system-level unmounts (ignore failures)
    for cmd in (["fusermount", "-u", mp], ["umount", mp]):
        try:
            subprocess.run(cmd, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except Exception:
            pass

    # 3) If the directory exists and is NOT a mount, remove it
    try:
        if os.path.isdir(mp) and not os.path.ismount(mp):
            shutil.rmtree(mp)
    except Exception:
        pass

    # 4) Recreate empty mountpoint
    os.makedirs(mp, exist_ok=True)

# Clean up any stale state, then mount
_safe_unmount()

from google.colab import drive
drive.mount(MOUNTPOINT, force_remount=True)

# Sanity check
assert os.path.isdir("/content/drive/MyDrive"), "Drive not mounted at /content/drive/MyDrive"
print("✅ Google Drive mounted and ready.")


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
✅ Google Drive mounted and ready.


In [7]:
# ============================================================
# A1 — Paths + latest silver resolver + load df (WarSpotting silver)
# ============================================================
import os, re
import pandas as pd
from glob import glob

DRIVE_ROOT = "/content/drive/MyDrive/osint_mvp"  # no trailing slash

paths = {
    "staged_dir": f"{DRIVE_ROOT}/staged",
    "out_dir":    f"{DRIVE_ROOT}/out",
    "plots":      f"{DRIVE_ROOT}/plots",
}

os.makedirs(paths["plots"], exist_ok=True)

# Find *latest* warspotting_norm_YYYY-MM-DD.csv in staged/
silver_candidates = sorted(glob(f"{paths['staged_dir']}/warspotting_norm_*.csv"))
if not silver_candidates:
    raise FileNotFoundError(
        f"No silver files found at {paths['staged_dir']}/warspotting_norm_*.csv.\n"
        "Run the ETL notebook first."
    )

silver_latest = silver_candidates[-1]
print("Using silver file:", silver_latest)

# Load per-loss normalized data (this is what analytics should use)
df = pd.read_csv(silver_latest, parse_dates=["date"])

# Derive a run label from filename for saving plots consistently
m = re.search(r"warspotting_norm_(\d{4}-\d{2}-\d{2})\.csv$", os.path.basename(silver_latest))
RUN_LABEL = m.group(1) if m else "latest"

print(f"Loaded {len(df)} rows. RUN_LABEL={RUN_LABEL}")
display(df.sample(min(3, len(df))))



In [ ]:
# ============================================================
# Block S1 — Daily spike detection (per class)
# Purpose:
#   - Quantify/visualize days with unusually high losses (e.g., ≥10).
#   - Save:
#       1) daily line with threshold overlay
#       2) monthly bar of "spike days"
# Config:
#   - target_classes: which platform classes to analyze
#   - SPIKE_THRESHOLD: "large assault" proxy (tune as needed)
# ============================================================

import os
import pandas as pd
import matplotlib.pyplot as plt

# --- Config ---
target_classes   = ["MBT", "IFV"]   # add more if you like
SPIKE_THRESHOLD  = 10               # try 8–12 to test sensitivity

# Ensure date is datetime (if not already)
df["date"] = pd.to_datetime(df["date"]).dt.date

def ensure_plots_dir():
    os.makedirs(paths["plots"], exist_ok=True)

def daily_counts_for_class(df_silver, cls):
    d = df_silver[df_silver["platform_class"] == cls].copy()
    if d.empty:
        return pd.DataFrame(columns=["date","n_daily"])
    g = (d.groupby("date", as_index=False)["count"].sum()
           .rename(columns={"count":"n_daily"}))
    g = g.sort_values("date")
    return g

def plot_spikes(df_daily, cls, threshold=SPIKE_THRESHOLD):
    if df_daily.empty:
        print(f"[S1] No data for {cls}")
        return
    ensure_plots_dir()

    # 1) Daily line with threshold overlay
    plt.figure(figsize=(10,4))
    plt.plot(df_daily["date"], df_daily["n_daily"], label=f"{cls} daily")
    plt.axhline(threshold, linestyle="--", label=f"spike≥{threshold}")
    plt.title(f"Daily {cls} losses with spike threshold ≥ {threshold}")
    plt.xlabel("Date"); plt.ylabel("Count per day"); plt.legend(); plt.tight_layout()
    out1 = os.path.join(paths["plots"], f"spikes_daily_{cls}_{RUN_ID}.png")
    plt.savefig(out1, dpi=160); plt.close()
    print("Saved:", out1)

    # 2) Monthly bar of spike-day counts
    m = pd.DataFrame(df_daily)
    m["date"] = pd.to_datetime(m["date"])
    m["is_spike"] = m["n_daily"] >= threshold
    monthly = (m.groupby(m["date"].dt.to_period("M"))["is_spike"]
                 .sum().rename("spike_days").reset_index())
    monthly["month"] = monthly["date"].astype(str)
    plt.figure(figsize=(8,4))
    plt.bar(monthly["month"], monthly["spike_days"])
    plt.title(f"Spike-day count by month — {cls} (≥{threshold})")
    plt.xlabel("Month"); plt.ylabel("# of spike days"); plt.tight_layout()
    out2 = os.path.join(paths["plots"], f"spikes_monthly_{cls}_{RUN_ID}.png")
    plt.savefig(out2, dpi=160); plt.close()
    print("Saved:", out2)

# ---- Run for each class ----
for cls in target_classes:
    s1_daily = daily_counts_for_class(df, cls)
    plot_spikes(s1_daily, cls, threshold=SPIKE_THRESHOLD)


Saved: /content/drive/MyDrive/osint_mvp/plots/spikes_daily_MBT_2025-09-08.png
Saved: /content/drive/MyDrive/osint_mvp/plots/spikes_monthly_MBT_2025-09-08.png
Saved: /content/drive/MyDrive/osint_mvp/plots/spikes_daily_IFV_2025-09-08.png
Saved: /content/drive/MyDrive/osint_mvp/plots/spikes_monthly_IFV_2025-09-08.png


In [ ]:
# ============================================================
# Block S2 — Rolling average losses over time
# Purpose:
#   - Show tempo shift by smoothing daily counts (e.g., 7/14-day).
#   - Save PNG per class.
# Notes:
#   - Rolling window on calendar days; handles sparse days by reindexing.
# ============================================================

import numpy as np

ROLLING_WINDOWS = [7, 14]      # tweak as you like
classes_for_ra  = ["MBT", "IFV"]

def daily_with_full_range(df_silver, cls):
    d = df_silver[df_silver["platform_class"] == cls].copy()
    if d.empty:
        return pd.DataFrame(columns=["date","n_daily"])
    d["date"] = pd.to_datetime(d["date"])
    g = (d.groupby("date", as_index=False)["count"].sum()
           .rename(columns={"count":"n_daily"}))
    # reindex to full continuous date range (fill missing days with 0)
    idx = pd.date_range(g["date"].min(), g["date"].max(), freq="D")
    g = g.set_index("date").reindex(idx).fillna(0.0).rename_axis("date").reset_index()
    g["n_daily"] = g["n_daily"].astype(int)
    return g

def plot_rolling_average(df_daily, cls, windows=ROLLING_WINDOWS):
    if df_daily.empty:
        print(f"[S2] No data for {cls}")
        return
    ensure_plots_dir()

    plt.figure(figsize=(10,4))
    # raw line (thin)
    plt.plot(df_daily["date"], df_daily["n_daily"], linewidth=1, label="daily")

    # rolling lines
    for w in windows:
        ra = df_daily["n_daily"].rolling(window=w, min_periods=max(1, w//2)).mean()
        plt.plot(df_daily["date"], ra, linewidth=2, label=f"{w}-day avg")

    plt.title(f"Rolling average losses — {cls}")
    plt.xlabel("Date"); plt.ylabel("Count per day"); plt.legend(); plt.tight_layout()
    out = os.path.join(paths["plots"], f"rolling_avg_{cls}_{RUN_ID}.png")
    plt.savefig(out, dpi=160); plt.close()
    print("Saved:", out)

for cls in classes_for_ra:
    ra_daily = daily_with_full_range(df, cls)
    plot_rolling_average(ra_daily, cls, windows=ROLLING_WINDOWS)


Saved: /content/drive/MyDrive/osint_mvp/plots/rolling_avg_MBT_2025-09-08.png
Saved: /content/drive/MyDrive/osint_mvp/plots/rolling_avg_IFV_2025-09-08.png


In [ ]:
# ============================================================
# Block S3 — Histogram of per-unit per-day losses
# Purpose:
#   - Use unit/day loss counts as a proxy for assault size/clustering.
#   - Shows if the tail (big per-unit/day events) is shrinking.
#   - Saves a PNG per class.
# Caveats:
#   - Relies on `unit_canonical` (or `unit_text`) being present for some rows.
#   - You can collapse to 'unit_text' if canonical is mostly null.
# ============================================================

# Choose which column to use for unit ID
UNIT_COL = "unit_canonical"     # fallback to "unit_text" if needed
classes_for_hist = ["MBT", "IFV"]

def unit_day_counts(df_silver, cls, unit_col=UNIT_COL):
    d = df_silver[(df_silver["platform_class"] == cls)].copy()
    d = d[d[unit_col].notna()]  # keep rows with unit info
    if d.empty:
        return pd.DataFrame(columns=["unit", "date", "n"])
    d["date"] = pd.to_datetime(d["date"]).dt.date
    g = (d.groupby([unit_col, "date"], as_index=False)["count"]
           .sum().rename(columns={unit_col:"unit", "count":"n"}))
    return g

def plot_unit_day_hist(g, cls):
    if g.empty:
        print(f"[S3] No unit/day data for {cls}")
        return
    ensure_plots_dir()

    plt.figure(figsize=(8,4))
    plt.hist(g["n"], bins=range(1, g["n"].max()+2), align="left")
    plt.title(f"Per-unit per-day loss distribution — {cls}")
    plt.xlabel("Losses per unit on a single day"); plt.ylabel("Frequency")
    plt.tight_layout()
    out = os.path.join(paths["plots"], f"hist_unit_day_{cls}_{RUN_ID}.png")
    plt.savefig(out, dpi=160); plt.close()
    print("Saved:", out)

for cls in classes_for_hist:
    g = unit_day_counts(df, cls, unit_col=UNIT_COL)
    plot_unit_day_hist(g, cls)


Saved: /content/drive/MyDrive/osint_mvp/plots/hist_unit_day_MBT_2025-09-08.png
Saved: /content/drive/MyDrive/osint_mvp/plots/hist_unit_day_IFV_2025-09-08.png


In [ ]:
# ============================================================
# Block G3 — Early vs Late density on a real basemap (shared scale)
# Purpose:
#   - Compare spatial concentration across two time windows fairly.
#   - Same extent, same gridsize, shared Log color scale.
# Outputs:
#   - plots/geo_hexbin_early_vs_late_<class>_<RUN_ID>.png
# Reqs:
#   - pip install geopandas contextily shapely pyproj (once per runtime)
# ============================================================

# 1) Install (safe to re-run)
!pip -q install geopandas contextily shapely pyproj >/dev/null

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, box
import contextily as cx
from matplotlib.colors import LogNorm

# --- Params you can tweak ---
target_class = "SPG"  # e.g. "MBT", "IFV", "MLRS", "SPG", "Truck"

early_start = "2025-06-01"
early_end   = "2025-07-07"

late_start  = "2025-07-08"
late_end    = "2025-09-07"

gridsize    = 60      # density resolution (bigger = coarser)
padding_m   = 40_000  # padding around combined extent in meters
plots_dir   = paths["plots"]  # from your earlier setup
os.makedirs(plots_dir, exist_ok=True)

# --- Helpers ---
def to_gdf_wgs84(df_):
    d = df_.copy()
    d = d[d["lat"].notna() & d["lon"].notna()]
    if d.empty:
        return gpd.GeoDataFrame(d, geometry=[], crs="EPSG:4326")
    return gpd.GeoDataFrame(d, geometry=gpd.points_from_xy(d["lon"], d["lat"]), crs="EPSG:4326")

def filter_window(df_, start, end, cls):
    mask_d = (pd.to_datetime(df_["date"]) >= pd.to_datetime(start)) & \
             (pd.to_datetime(df_["date"]) <= pd.to_datetime(end))
    mask_c = (df_["platform_class"] == cls)
    return df_.loc[mask_d & mask_c].copy()

# --- Build GeoDataFrames for the two windows (WGS84) ---
df_early = filter_window(df, early_start, early_end, target_class)
df_late  = filter_window(df, late_start,  late_end,  target_class)

gdf_e_wgs = to_gdf_wgs84(df_early)
gdf_l_wgs = to_gdf_wgs84(df_late)

# Limit roughly to Ukraine to avoid stray points; adjust if you like
UKR_BBOX_WGS84 = box(22.0, 44.0, 41.5, 52.5)
gdf_e_wgs = gdf_e_wgs[gdf_e_wgs.geometry.within(UKR_BBOX_WGS84)]
gdf_l_wgs = gdf_l_wgs[gdf_l_wgs.geometry.within(UKR_BBOX_WGS84)]

if gdf_e_wgs.empty and gdf_l_wgs.empty:
    print(f"[G3] No geo-tagged points for class '{target_class}' in either window.")
else:
    # Project to Web Mercator for tiles
    gdf_e = gdf_e_wgs.to_crs(epsg=3857)
    gdf_l = gdf_l_wgs.to_crs(epsg=3857)

    # Compute a combined plot extent (same zoom for both panels)
    xs = np.concatenate([gdf_e.geometry.x.values if not gdf_e.empty else np.array([]),
                         gdf_l.geometry.x.values if not gdf_l.empty else np.array([])])
    ys = np.concatenate([gdf_e.geometry.y.values if not gdf_e.empty else np.array([]),
                         gdf_l.geometry.y.values if not gdf_l.empty else np.array([])])

    if len(xs) == 0:  # one window might be empty, the other not; fit to the non-empty one
        xs = gdf_e.geometry.x.values if not gdf_e.empty else gdf_l.geometry.x.values
        ys = gdf_e.geometry.y.values if not gdf_e.empty else gdf_l.geometry.y.values

    x_min, x_max = xs.min()-padding_m, xs.max()+padding_m
    y_min, y_max = ys.min()-padding_m, ys.max()+padding_m
    extent = (x_min, x_max, y_min, y_max)

    # Determine shared color normalization using 2D histograms (counts)
    def counts_max(gdf_, extent, gridsize):
        if gdf_.empty:
            return 0
        x = gdf_.geometry.x.values
        y = gdf_.geometry.y.values
        H, _, _ = np.histogram2d(x, y, bins=gridsize, range=[[extent[0], extent[1]], [extent[2], extent[3]]])
        return int(H.max())

    max_e = counts_max(gdf_e, extent, gridsize)
    max_l = counts_max(gdf_l, extent, gridsize)
    vmax  = max(max_e, max_l, 1)  # at least 1 to keep LogNorm happy

    # Plot two panels with shared Log color scale
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 6), constrained_layout=True, sharex=True, sharey=True)

    def hexbin_panel(ax, gdf_, title):
        if gdf_.empty:
            ax.text(0.5, 0.5, "No data", ha="center", va="center", fontsize=12, transform=ax.transAxes)
        else:
            x = gdf_.geometry.x.values
            y = gdf_.geometry.y.values
            hb = ax.hexbin(x, y, gridsize=gridsize, mincnt=1,
                           extent=[extent[0], extent[1], extent[2], extent[3]],
                           norm=LogNorm(vmin=1, vmax=vmax))
            cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs=gdf_.crs)
        ax.set_title(title)
        ax.set_xlim(extent[0], extent[1])
        ax.set_ylim(extent[2], extent[3])
        ax.set_axis_off()
        return hb if not gdf_.empty else None

    hb1 = hexbin_panel(ax1, gdf_e, f"Early: {early_start} → {early_end}")
    hb2 = hexbin_panel(ax2, gdf_l, f"Late:  {late_start} → {late_end}")

    # Shared colorbar (if at least one panel has data)
    mappable = hb1 or hb2
    if mappable:
        cbar = fig.colorbar(mappable, ax=[ax1, ax2], shrink=0.85)
        cbar.set_label("Loss density (log scale)")

    fig.suptitle(f"{target_class} losses — spatial density (Early vs Late)", y=1.02, fontsize=14)

    outpath = os.path.join(plots_dir, f"geo_hexbin_early_vs_late_{target_class}_{RUN_ID}.png")
    plt.savefig(outpath, dpi=170, bbox_inches="tight", pad_inches=0.05)
    plt.close()
    print("Saved:", outpath)


Saved: /content/drive/MyDrive/osint_mvp/plots/geo_hexbin_early_vs_late_SPG_2025-09-08.png


In [ ]:
# ============================================================
# Block G4 (fixed) — Smooth KDE heatmap on a real basemap
# Draw basemap FIRST, then heatmap on top (with alpha + zorder)
# ============================================================

# 1) Install deps (safe to re-run)
!pip -q install geopandas contextily shapely pyproj scikit-learn >/dev/null

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import contextily as cx
from sklearn.neighbors import KernelDensity

# --- Params you can tweak ---
target_class = "SPG"       # e.g., "MBT", "IFV", "MLRS", "SPG", "Truck"
use_date_window = False    # set True to restrict by date
start_date = "2025-07-01"
end_date   = "2025-08-15"

bandwidth_m = 25_000       # 15k–40k looks good
grid_px     = 900          # width in pixels
cmap_name   = "inferno"
alpha_heat  = 0.65
padding_m   = 40_000

plots_dir = paths["plots"]
os.makedirs(plots_dir, exist_ok=True)

# --- Quick sanity: what classes do we have? ---
print("Classes present:", sorted(df["platform_class"].dropna().unique())[:20])

# Filter rows (class + optional date + lat/lon present)
df_in = df[(df["platform_class"] == target_class) &
           df["lat"].notna() & df["lon"].notna()].copy()

if use_date_window:
    mask = (pd.to_datetime(df_in["date"]) >= pd.to_datetime(start_date)) & \
           (pd.to_datetime(df_in["date"]) <= pd.to_datetime(end_date))
    df_in = df_in.loc[mask]

print(f"{len(df_in)} points for class={target_class}"
      + (f" in {start_date}→{end_date}" if use_date_window else ""))

if df_in.empty:
    print(f"[G4] No data for class '{target_class}' with current filters.")
else:
    # GeoDataFrame WGS84 → clip to Ukraine bbox → project to Web Mercator
    gdf_wgs = gpd.GeoDataFrame(
        df_in, geometry=gpd.points_from_xy(df_in["lon"], df_in["lat"]), crs="EPSG:4326"
    )
    UKR_BBOX_WGS84 = box(22.0, 44.0, 41.5, 52.5)
    gdf_wgs = gdf_wgs[gdf_wgs.geometry.within(UKR_BBOX_WGS84)]
    if gdf_wgs.empty:
        print(f"[G4] No points inside Ukraine bbox for class '{target_class}'.")
    else:
        gdf = gdf_wgs.to_crs(epsg=3857)
        xs = gdf.geometry.x.values
        ys = gdf.geometry.y.values

        # Extent with padding
        x_min, x_max = xs.min() - padding_m, xs.max() + padding_m
        y_min, y_max = ys.min() - padding_m, ys.max() + padding_m

        # Grid for KDE
        width = grid_px
        aspect = (y_max - y_min) / (x_max - x_min)
        height = max(1, int(width * aspect))
        xlin = np.linspace(x_min, x_max, width)
        ylin = np.linspace(y_min, y_max, height)
        xx, yy = np.meshgrid(xlin, ylin)
        grid_points = np.vstack([xx.ravel(), yy.ravel()]).T

        # KDE
        kde = KernelDensity(bandwidth=bandwidth_m, kernel="gaussian")
        kde.fit(np.column_stack([xs, ys]))
        log_d = kde.score_samples(grid_points)
        dens = np.exp(log_d).reshape(height, width)

        # Robust scaling (avoid all-zero look)
        vmax_quant = np.quantile(dens, 0.99)
        vmax_direct = dens.max()
        vmax = max(vmax_quant, vmax_direct * 0.7, 1e-12)
        dens_scaled = np.clip(dens, 0, vmax) / vmax

        # ---- Plot: basemap first, then heatmap on top ----
        fig, ax = plt.subplots(figsize=(9, 7))

        # Basemap (goes underneath)
        # Use the full extent; contextily respects current CRS (EPSG:3857)
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:3857")

        # Heatmap on top (semi-transparent)
        im = ax.imshow(
            np.flipud(dens_scaled),
            extent=[x_min, x_max, y_min, y_max],
            cmap=cmap_name, alpha=alpha_heat, zorder=5
        )

        # (Optional) a few points on top as dots, to sanity-check alignment
        if len(xs) > 0:
            ax.scatter(xs[::max(1, len(xs)//300)], ys[::max(1, len(ys)//300)],
                       s=6, alpha=0.6, zorder=6)

        title = f"KDE heatmap — {target_class} losses"
        if use_date_window:
            title += f" ({start_date} → {end_date})"
        ax.set_title(title)
        ax.set_axis_off()

        cbar = fig.colorbar(im, ax=ax, shrink=0.85)
        cbar.set_label("Relative density (scaled)")

        outpath = os.path.join(plots_dir, f"geo_kde_heatmap_{target_class}_{RUN_ID}.png")
        plt.savefig(outpath, dpi=170, bbox_inches="tight", pad_inches=0.05)
        plt.close()
        print("Saved:", outpath)


Classes present: ['AD', 'Airplanes', 'Ambulances, medical vehicles', 'C2', 'Engineering', 'Helicopter', 'IFV', 'Infantry mobility vehicles', 'MBT', 'MLRS', 'Other', 'Radars, jammers', 'SPG', 'TowedArtillery', 'Truck', 'UAV']
11 points for class=SPG
Saved: /content/drive/MyDrive/osint_mvp/plots/geo_kde_heatmap_SPG_2025-09-08.png


In [ ]:
# ============================================================
# Block G4b — Adaptive KDE heatmaps on a real basemap (multi-class)
# Purpose:
#   - Make "high-signal" heatmaps without manual parameter hunting.
#   - One PNG per class. Uses adaptive bandwidth + robust color scaling.
# Outputs:
#   - plots/geo_kde_heatmap_<class>_<RUN_ID>.png
# Reqs (once per runtime):
#   - pip install geopandas contextily shapely pyproj scikit-learn
# ============================================================

# 1) Install deps (safe to re-run)
!pip -q install geopandas contextily shapely pyproj scikit-learn >/dev/null

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import contextily as cx
from sklearn.neighbors import KernelDensity

# --- Pick classes you want maps for ---
target_classes = ["MBT", "IFV", "SPG", "MLRS", "Truck"]  # edit as you like

# --- Optional: date window (set to True to restrict) ---
USE_DATE_WINDOW = False
DATE_START = "2025-06-01"
DATE_END   = "2025-09-07"

# --- Render controls (sane defaults) ---
GRID_PX    = 1100        # output width (px); height auto-kept by aspect
ALPHA_HEAT = 0.68        # heat overlay opacity
CMAP_NAME  = "inferno"   # "magma", "plasma", "viridis" also nice
PAD_M      = 60_000      # padding around data extent in meters
UKR_CLIP   = True        # True = clip to coarse Ukraine bbox; set False to show all

# Paths
plots_dir = paths["plots"]  # from your earlier setup
os.makedirs(plots_dir, exist_ok=True)

# --- Coarse Ukraine bbox (WGS84) ---
UKR_BBOX_WGS84 = box(22.0, 44.0, 41.5, 52.5)

# --- Helpers ------------------------------------------------
def _filter_df(df_in, cls):
    d = df_in[(df_in["platform_class"] == cls) & df_in["lat"].notna() & df_in["lon"].notna()].copy()
    if USE_DATE_WINDOW:
        mask = (pd.to_datetime(d["date"]) >= pd.to_datetime(DATE_START)) & \
               (pd.to_datetime(d["date"]) <= pd.to_datetime(DATE_END))
        d = d.loc[mask]
    return d

def _adaptive_bandwidth_m(x, y):
    """
    Scott-like 2D bandwidth in meters with safeguards:
    h = c * min(std_x, IQR_x/1.34, std_y, IQR_y/1.34) * n^(-1/6)
    then clamp to [12.5 km, 45 km] so it looks good for slide scale.
    """
    n = max(len(x), 1)
    sx = np.std(x) if n > 1 else 0.0
    sy = np.std(y) if n > 1 else 0.0
    qx = np.subtract(*np.percentile(x, [75, 25])) if n > 1 else 0.0
    qy = np.subtract(*np.percentile(y, [75, 25])) if n > 1 else 0.0
    sig = np.nanmax([sx, sy, qx/1.34 if qx>0 else 0.0, qy/1.34 if qy>0 else 0.0])
    if sig <= 0:
        return 20_000  # fallback
    h = 1.06 * sig * (n ** (-1/6))  # Scott factor for 2D
    return float(np.clip(h, 12_500, 45_000))

def _robust_scale(dens):
    """Scale to 99th percentile so maps aren't washed out by a few hot pixels."""
    if dens.size == 0:
        return dens, 1.0
    vmax = np.quantile(dens, 0.99)
    if vmax <= 0:
        return dens, 1.0
    return np.clip(dens, 0, vmax) / vmax, vmax

def _make_heatmap_for_class(df_base, cls):
    d = _filter_df(df_base, cls)
    print(f"[G4b] {cls}: {len(d)} rows before bbox clip/date window")
    if d.empty:
        print(f"[G4b] Skipping {cls}: no rows.")
        return

    # WGS84 → (optional) bbox clip → Web Mercator
    gdf_wgs = gpd.GeoDataFrame(d, geometry=gpd.points_from_xy(d["lon"], d["lat"]), crs="EPSG:4326")
    if UKR_CLIP:
        gdf_wgs = gdf_wgs[gdf_wgs.geometry.within(UKR_BBOX_WGS84)]
        print(f"[G4b] {cls}: {len(gdf_wgs)} rows after Ukraine bbox clip")
        if gdf_wgs.empty:
            print(f"[G4b] Skipping {cls}: empty after bbox clip. Try UKR_CLIP=False.")
            return

    gdf = gdf_wgs.to_crs(epsg=3857)
    xs = gdf.geometry.x.values
    ys = gdf.geometry.y.values

    # Extent + padding
    x_min, x_max = xs.min() - PAD_M, xs.max() + PAD_M
    y_min, y_max = ys.min() - PAD_M, ys.max() + PAD_M

    # KDE grid
    width = GRID_PX
    aspect = (y_max - y_min) / (x_max - x_min)
    height = max(1, int(width * aspect))
    xlin = np.linspace(x_min, x_max, width)
    ylin = np.linspace(y_min, y_max, height)
    xx, yy = np.meshgrid(xlin, ylin)
    grid_points = np.vstack([xx.ravel(), yy.ravel()]).T

    # Adaptive bandwidth
    bw = _adaptive_bandwidth_m(xs, ys)
    kde = KernelDensity(bandwidth=bw, kernel="gaussian")
    kde.fit(np.column_stack([xs, ys]))
    dens = np.exp(kde.score_samples(grid_points)).reshape(height, width)

    # Robust scale to 99th pct
    dens_scaled, vmax = _robust_scale(dens)

    # --- Plot: basemap first, then heatmap + sparse dots ---
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.set_xlim(x_min, x_max); ax.set_ylim(y_min, y_max)
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs="EPSG:3857")

    im = ax.imshow(
        np.flipud(dens_scaled),
        extent=[x_min, x_max, y_min, y_max],
        cmap=CMAP_NAME, alpha=ALPHA_HEAT, zorder=5
    )

    # Sparse dot overlay (sample up to ~400 points for sanity)
    step = max(1, len(xs)//400)
    ax.scatter(xs[::step], ys[::step], s=6, alpha=0.6, zorder=6)

    title = f"KDE heatmap — {cls}"
    if USE_DATE_WINDOW:
        title += f" ({DATE_START} → {DATE_END})"
    ax.set_title(title)
    ax.set_axis_off()

    cbar = fig.colorbar(im, ax=ax, shrink=0.85)
    cbar.set_label("Relative density (scaled to 99th pct)")

    out = os.path.join(plots_dir, f"geo_kde_heatmap_{cls}_{RUN_ID}.png")
    plt.savefig(out, dpi=175, bbox_inches="tight", pad_inches=0.05)
    plt.close()
    print(f"[G4b] Saved: {out} (n={len(xs)}, bw≈{int(bw)} m)")

# ---- Run for each class ----
for cls in target_classes:
    _make_heatmap_for_class(df, cls)


[G4b] MBT: 35 rows before bbox clip/date window
[G4b] MBT: 35 rows after Ukraine bbox clip
[G4b] Saved: /content/drive/MyDrive/osint_mvp/plots/geo_kde_heatmap_MBT_2025-09-08.png (n=35, bw≈45000 m)
[G4b] IFV: 95 rows before bbox clip/date window
[G4b] IFV: 95 rows after Ukraine bbox clip
[G4b] Saved: /content/drive/MyDrive/osint_mvp/plots/geo_kde_heatmap_IFV_2025-09-08.png (n=95, bw≈45000 m)
[G4b] SPG: 11 rows before bbox clip/date window
[G4b] SPG: 11 rows after Ukraine bbox clip
[G4b] Saved: /content/drive/MyDrive/osint_mvp/plots/geo_kde_heatmap_SPG_2025-09-08.png (n=11, bw≈45000 m)
[G4b] MLRS: 16 rows before bbox clip/date window
[G4b] MLRS: 16 rows after Ukraine bbox clip
[G4b] Saved: /content/drive/MyDrive/osint_mvp/plots/geo_kde_heatmap_MLRS_2025-09-08.png (n=16, bw≈45000 m)
[G4b] Truck: 39 rows before bbox clip/date window
[G4b] Truck: 39 rows after Ukraine bbox clip
[G4b] Saved: /content/drive/MyDrive/osint_mvp/plots/geo_kde_heatmap_Truck_2025-09-08.png (n=39, bw≈45000 m)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
# ============================================================
# Block S4 — Mass vehicle loss spike events over time
# Purpose:
#   - Count how often same-day, same-location losses > threshold
#   - Plot per-week or per-month frequency
# Output:
#   - PNG bar chart in plots/ (mass_loss_spikes_<class>.png)
# Config:
#   - platform_class: "MBT", "IFV", etc.
#   - threshold: what counts as a spike
# ============================================================

import matplotlib.pyplot as plt
import os

# --- Config ---
platform_class = "MBT"        # Try "IFV" or "SPG" too
spike_threshold = 6           # Same-day + location losses > 6 = spike
group_by = "W"                # "W" = week, "M" = month
plots_dir = paths["plots"]
os.makedirs(plots_dir, exist_ok=True)

# --- Filter + clean ---
df_spike = df[(df["platform_class"] == platform_class) &
              df["date"].notna() & df["location"].notna()].copy()
df_spike["date"] = pd.to_datetime(df_spike["date"])

# Group by (date + location) to count losses per day/location
grp = (df_spike.groupby(["date", "location"], as_index=False)["count"]
       .sum().rename(columns={"count": "n_losses"}))

# Flag "spike events"
grp["is_spike"] = grp["n_losses"] > spike_threshold

# Group by week/month: how many spike events?
grp["period"] = grp["date"].dt.to_period(group_by)
summary = grp[grp["is_spike"]].groupby("period").size().reset_index(name="n_spikes")
summary["period"] = summary["period"].astype(str)

# Optional: fill missing periods
all_periods = pd.period_range(grp["period"].min(), grp["period"].max(), freq=group_by)
summary = summary.set_index("period").reindex(all_periods.astype(str), fill_value=0).reset_index()

# --- Plot ---
plt.figure(figsize=(10,5))
plt.bar(summary["index"], summary["n_spikes"])
plt.title(f"{platform_class} mass-loss events over time\n(>{spike_threshold} in 1 day/region)")
plt.xlabel("Time"); plt.ylabel("# of spike events")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

# --- Save ---
outpath = os.path.join(plots_dir, f"mass_loss_spikes_{platform_class}_{RUN_ID}.png")
plt.savefig(outpath, dpi=160)
plt.close()
print("Saved:", outpath)


Saved: /content/drive/MyDrive/osint_mvp/plots/mass_loss_spikes_MBT_2025-09-08.png


In [11]:
# ============================================================
# Block S5 — Mass loss spikes by unit (stacked by platform class)
# Spike event: same day + same unit + platform_class with > threshold losses
# Depends on:
#   - df (loaded in A1, warspotting_norm_*.csv)
#   - RUN_LABEL, paths["plots"] (set in A1)
# Outputs:
#   - plots/mass_loss_unit_spikes_by_class_<RUN_LABEL>.png
#   - out/spike_events_by_unit_<RUN_LABEL>.csv (optional detail export)
# ============================================================

import os
import pandas as pd
import matplotlib.pyplot as plt

# --- Config ---
spike_threshold = 6            # >6 losses = spike
group_by = "W"                 # "W" weekly, "M" monthly
min_unit_len = 3               # ignore very short unit labels
export_csv = True              # also save a CSV of the spike events

# --- Sanity: required globals from A1 ---
assert "df" in globals(), "df is not loaded. Run A1 first."
assert "paths" in globals() and "plots" in paths, "paths['plots'] missing. Run A1 first."
RUN_LABEL = globals().get("RUN_LABEL", "latest")

os.makedirs(paths["plots"], exist_ok=True)
os.makedirs(paths.get("out_dir", os.path.join(paths["plots"], "..", "out")), exist_ok=True)

# --- Prep & unit fallback ---
df_unit = df.copy()
df_unit["date"] = pd.to_datetime(df_unit["date"], errors="coerce")

# create a unit column with fallback: unit_canonical -> unit_text
if "unit_canonical" in df_unit.columns:
    unit_series = df_unit["unit_canonical"]
else:
    unit_series = pd.Series([None] * len(df_unit))

if unit_series.isna().all() and "unit_text" in df_unit.columns:
    unit_series = df_unit["unit_text"]

df_unit["unit"] = unit_series.astype("string").str.strip()
df_unit.loc[df_unit["unit"].isin(["", "None", "nan"]), "unit"] = pd.NA

# filter rows usable for this analysis
pre_rows = len(df_unit)
df_unit = df_unit[
    df_unit["date"].notna()
    & df_unit["platform_class"].notna()
    & df_unit["unit"].notna()
    & (df_unit["unit"].str.len() >= min_unit_len)
].copy()
post_rows = len(df_unit)

print(f"[S5] Usable rows with unit info: {post_rows}/{pre_rows} "
      f"({post_rows/pre_rows*100:.1f}%)")

if df_unit.empty:
    print("[S5] No rows with unit/platform_class/date; nothing to plot.")
else:
    # normalize count column presence
    if "count" not in df_unit.columns:
        df_unit["count"] = 1

    # losses per (date, unit, platform_class)
    grp = (df_unit
           .groupby(["date", "unit", "platform_class"], as_index=False)["count"]
           .sum()
           .rename(columns={"count": "n_losses"}))

    # flag spikes
    grp["is_spike"] = grp["n_losses"] > spike_threshold

    # period bucketing
    if group_by not in {"W", "M"}:
        raise ValueError("group_by must be 'W' (weekly) or 'M' (monthly).")
    grp["period"] = grp["date"].dt.to_period(group_by).astype(str)

    spikes = grp[grp["is_spike"]].copy()
    if spikes.empty:
        print(f"[S5] No spike events (> {spike_threshold}) found.")
    else:
        # count spikes per period & platform class
        spike_counts = (spikes
                        .groupby(["period", "platform_class"])
                        .size()
                        .reset_index(name="n_spikes"))

        # pivot for stacked bars; reindex to continuous periods
        pivot = spike_counts.pivot(index="period",
                                   columns="platform_class",
                                   values="n_spikes").fillna(0)

        all_periods = pd.period_range(pivot.index.min(), pivot.index.max(),
                                      freq=group_by).astype(str)
        pivot = pivot.reindex(all_periods, fill_value=0)

        # consistent column order
        pivot = pivot[sorted(pivot.columns)]

        # plot
        fig, ax = plt.subplots(figsize=(11, 6))
        bottom = None
        for col in pivot.columns:
            ax.bar(pivot.index, pivot[col], label=col, bottom=bottom)
            bottom = (pivot[col] if bottom is None else bottom + pivot[col])

        ax.set_title(f"Same‑unit daily spike events (> {spike_threshold} losses)\n"
                     f"Stacked by platform class")
        ax.set_xlabel("Time"); ax.set_ylabel("# of spike events")
        plt.xticks(rotation=45, ha="right")
        ax.legend(ncol=3, fontsize=9)
        plt.tight_layout()

        out_png = os.path.join(paths["plots"],
                               f"mass_loss_unit_spikes_by_class_{RUN_LABEL}.png")
        plt.savefig(out_png, dpi=160)
        plt.close()
        print("Saved plot:", out_png)

        # optional: export detailed spike table for QA / slide notes
        if export_csv:
            out_csv = os.path.join(paths.get("out_dir", f"{os.path.dirname(paths['plots'])}/out"),
                                   f"spike_events_by_unit_{RUN_LABEL}.csv")
            spikes.sort_values(["date", "unit", "platform_class", "n_losses"]) \
                  .to_csv(out_csv, index=False)
            print("Saved spike events CSV:", out_csv)


[S5] No spike events (> 6) found.


In [ ]:
# ============================================================
# BLOCK U1+ — Interactive per‑unit map (Folium) with basemap selector
# What it does:
#   - Lets you pick a unit (free-text contains match) and a basemap
#   - Plots one marker per loss (MBT/IFV/APC/BMP by default)
#   - Tooltip shows model, class, unit, date, nearest location
#   - Outputs an HTML map to /plots/
# Depends on:
#   - df (loaded in A1), paths["plots"], RUN_LABEL
# ============================================================

import os
import re
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from datetime import datetime

# ---------- Config you can tweak ----------
DEFAULT_FOCUS_CLASSES = {"MBT", "IFV", "APC", "BMP"}  # set() for all classes
DATE_START = "2025-06-01"
DATE_END   = "2025-09-30"  # extend if you have newer data
UNIT_QUERY = ""            # e.g., "236th Artillery", "752nd", leave "" for no unit filter
BASEMAP    = "Stamen Terrain"  # choices: "OpenStreetMap", "Stamen Terrain", "OpenTopoMap", "Esri Satellite"
USE_CLASS_COLORS = True    # color markers by platform_class instead of all red
# -----------------------------------------

# Guards
assert "df" in globals(), "df not loaded. Run A1 first."
assert "paths" in globals() and "plots" in paths, "paths['plots'] missing. Run A1 first."
RUN_LABEL = globals().get("RUN_LABEL", "latest")
os.makedirs(paths["plots"], exist_ok=True)

# Basemap registry
BASEMAPS = {
    "OpenStreetMap": dict(tiles="OpenStreetMap", attr=None),
    "Stamen Terrain": dict(tiles="Stamen Terrain", attr=None),
    "OpenTopoMap": dict(
        tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
        attr="Map data: © OpenStreetMap contributors, SRTM | Map style: © OpenTopoMap (CC-BY-SA)"
    ),
    "Esri Satellite": dict(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        attr="Tiles © Esri"
    ),
}

if BASEMAP not in BASEMAPS:
    raise ValueError(f"Unknown BASEMAP '{BASEMAP}'. Choose one of: {list(BASEMAPS)}")

# Prepare data with unit fallback
d = df.copy()
d["date"] = pd.to_datetime(d["date"], errors="coerce")

unit_series = d["unit_canonical"] if "unit_canonical" in d.columns else pd.Series([None]*len(d))
if (unit_series.isna().all() or (unit_series.astype(str).str.strip() == "").all()) and "unit_text" in d.columns:
    unit_series = d["unit_text"]
d["unit"] = unit_series.astype("string").str.strip()
d.loc[d["unit"].isin(["", "None", "nan"]), "unit"] = pd.NA

# Filters
mask = d["date"].between(DATE_START, DATE_END) & d["lat"].notna() & d["lon"].notna()
if DEFAULT_FOCUS_CLASSES:
    mask &= d["platform_class"].isin(DEFAULT_FOCUS_CLASSES)
if UNIT_QUERY:
    q = UNIT_QUERY.strip().lower()
    mask &= d["unit"].fillna("").str.lower().str.contains(q)

dm = d[mask].copy()

if dm.empty:
    print("[U1+] No rows match current filters. Try loosening UNIT_QUERY, classes, or dates.")
else:
    # Class color palette (simple & readable)
    class_colors = {
        "MBT": "#e41a1c",   # red
        "IFV": "#377eb8",   # blue
        "APC": "#4daf4a",   # green
        "BMP": "#984ea3",   # purple
        "SPG": "#ff7f00",   # orange
        "MLRS":"#a65628",   # brown
        "Truck":"#999999",  # gray
    }
    def marker_color(cls):
        if not USE_CLASS_COLORS:
            return "red"
        return class_colors.get(cls, "#555555")

    # Center on mean of points
    center_lat = dm["lat"].mean()
    center_lon = dm["lon"].mean()

    # Build map
    bm = BASEMAPS[BASEMAP]
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6,
                   tiles=bm["tiles"], attr=bm["attr"])

    # Add a layer control and (optional) also include OSM for quick switching
    if BASEMAP != "OpenStreetMap":
        folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(m)
    if BASEMAP != "Stamen Terrain":
        folium.TileLayer("Stamen Terrain", name="Stamen Terrain").add_to(m)
    if BASEMAP != "OpenTopoMap":
        folium.TileLayer(
            tiles=BASEMAPS["OpenTopoMap"]["tiles"],
            attr=BASEMAPS["OpenTopoMap"]["attr"],
            name="OpenTopoMap"
        ).add_to(m)
    if BASEMAP != "Esri Satellite":
        folium.TileLayer(
            tiles=BASEMAPS["Esri Satellite"]["tiles"],
            attr=BASEMAPS["Esri Satellite"]["attr"],
            name="Esri Satellite"
        ).add_to(m)

    cluster = MarkerCluster(name="Losses").add_to(m)

    # Add one marker per loss
    for _, r in dm.sort_values("date").iterrows():
        unit_display = r.get("unit") or "Unknown unit"
        label = f"""
        <b>{r.get('model','Unknown')}</b><br>
        {r.get('platform_class','')}<br>
        <i>{unit_display}</i><br>
        <b>{r['date'].date()}</b><br>
        <small>{r.get('location','')}</small>
        """
        folium.CircleMarker(
            location=(r["lat"], r["lon"]),
            radius=4,
            color=marker_color(r.get("platform_class")),
            fill=True,
            fill_opacity=0.75,
            tooltip=folium.Tooltip(label, sticky=True),
        ).add_to(cluster)

    folium.LayerControl(collapsed=True).add_to(m)

    # Output file name with a tidy unit slug
    def slugify(s):
        if not s:
            return "all-units"
        s = re.sub(r"[^a-zA-Z0-9]+", "-", s.strip())
        return re.sub(r"-+", "-", s).strip("-").lower()

    unit_slug = slugify(UNIT_QUERY) if UNIT_QUERY else "all-units"
    out_html = os.path.join(paths["plots"], f"map_per_loss_{unit_slug}_{BASEMAP.replace(' ','_')}_{RUN_LABEL}.html")
    m.save(out_html)

    print(f"✅ Saved interactive map: {out_html}")
    print(f"Filters → unit: '{UNIT_QUERY or 'ALL'}', classes: {sorted(DEFAULT_FOCUS_CLASSES) if DEFAULT_FOCUS_CLASSES else 'ALL'}, "
          f"dates: {DATE_START} → {DATE_END}, basemap: {BASEMAP}")
